<a href="https://colab.research.google.com/github/Asher-Topaz/AIGC5500_advanced_deep_learning/blob/main/lab3_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np
torch.manual_seed(7)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 20
BATCH_SIZE = 64

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
trainset = MNIST(root='./pt_data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=True)
data = next(iter(trainloader))
mean = data[0].mean()
stddev = data[0].std()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 34911550.17it/s]


Extracting ./pt_data/MNIST/raw/train-images-idx3-ubyte.gz to ./pt_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1411180.03it/s]


Extracting ./pt_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./pt_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9631060.79it/s]


Extracting ./pt_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./pt_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3707049.77it/s]

Extracting ./pt_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./pt_data/MNIST/raw



In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, stddev),
     transforms.Lambda(lambda x: (x - mean) / stddev)])

In [ ]:
trainset = MNIST(root='./pt_data', train=True, download=True, transform=transform)
testset = MNIST(root='./pt_data', train=False, download=True, transform=transform)

In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.BatchNorm1d(784),
    nn.Linear(784, 25),
    nn.BatchNorm1d(25),
    nn.ReLU(),
    nn.Linear(25, 10)
)

In [ ]:
nn.init.kaiming_normal_(model[2].weight)
nn.init.constant_(model[2].bias, 0.0)

nn.init.xavier_uniform_(model[5].weight)
nn.init.constant_(model[5].bias, 0.0)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [ ]:
learning_rate = 0.001

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

model.to(device)


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Linear(in_features=784, out_features=25, bias=True)
  (3): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): ReLU()
  (5): Linear(in_features=25, out_features=10, bias=True)
)

In [ ]:
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
for i in range(EPOCHS):
    model.train() # Set model in training mode.
    train_loss = 0.0
    train_correct = 0
    train_batches = 0
    for inputs, targets in trainloader:
        # Move data to GPU.
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the parameter gradients.
        optimizer.zero_grad()

        # Forward pass.
        outputs = model(inputs)
        # Cross-entropy loss does not need one-hot targets in PyTorch.
        loss = loss_function(outputs, targets)

        # Accumulate metrics.
        _, indices = torch.max(outputs.data, 1)
        train_correct += (indices == targets).sum().item()
        train_batches +=  1
        train_loss += loss.item()

        # Backward pass and update.
        loss.backward()
        optimizer.step()

    train_loss = train_loss / train_batches
    train_acc = train_correct / (train_batches * BATCH_SIZE)

    # Evaluate the model on the test dataset. Identical to loop above but without
    # weight adjustment.
    model.eval() # Set model in inference mode.
    test_loss = 0.0
    test_correct = 0
    test_batches = 0
    for inputs, targets in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        _, indices = torch.max(outputs, 1)
        test_correct += (indices == targets).sum().item()
        test_batches +=  1
        test_loss += loss.item()

    test_loss = test_loss / test_batches
    test_acc = test_correct / (test_batches * BATCH_SIZE)

    print(f'Epoch {i+1}/{EPOCHS} loss: {train_loss:.4f} - acc: {train_acc:0.4f} - val_loss: {test_loss:.4f} - val_acc: {test_acc:0.4f}')

Epoch 1/20 loss: 0.4875 - acc: 0.8765 - val_loss: 0.5526 - val_acc: 0.9213
Epoch 2/20 loss: 0.2354 - acc: 0.9323 - val_loss: 0.4479 - val_acc: 0.9331
Epoch 3/20 loss: 0.1907 - acc: 0.9435 - val_loss: 0.6850 - val_acc: 0.9394
Epoch 4/20 loss: 0.1663 - acc: 0.9495 - val_loss: 0.6992 - val_acc: 0.9440
Epoch 5/20 loss: 0.1496 - acc: 0.9548 - val_loss: 0.7639 - val_acc: 0.9429
Epoch 6/20 loss: 0.1368 - acc: 0.9587 - val_loss: 0.8864 - val_acc: 0.9420
Epoch 7/20 loss: 0.1271 - acc: 0.9606 - val_loss: 0.9551 - val_acc: 0.9412
Epoch 8/20 loss: 0.1207 - acc: 0.9628 - val_loss: 0.7198 - val_acc: 0.9462
Epoch 9/20 loss: 0.1149 - acc: 0.9647 - val_loss: 1.0243 - val_acc: 0.9445
Epoch 10/20 loss: 0.1083 - acc: 0.9662 - val_loss: 0.9439 - val_acc: 0.9476
Epoch 11/20 loss: 0.1037 - acc: 0.9671 - val_loss: 1.1389 - val_acc: 0.9475
Epoch 12/20 loss: 0.0992 - acc: 0.9687 - val_loss: 1.6287 - val_acc: 0.9476
Epoch 13/20 loss: 0.0972 - acc: 0.9688 - val_loss: 1.5402 - val_acc: 0.9447
Epoch 14/20 loss: 0.0